In [3]:
import numpy as np
import matplotlib.pyplot as plt
from elastica.modules import BaseSystemCollection, Constraints, Forcing, Damping 

from elastica.rod.cosserat_rod import CosseratRod 
from elastica.dissipation import AnalyticalLinearDamper
from elastica.boundary_conditions import OneEndFixedRod
from elastica.external_forces import EndpointForces, GravityForces 
from elastica import Connections
from elastica import FixedJoint
from elastica.callback_functions import CallBackBaseClass
from elastica.timestepper import integrate

from elastica.timestepper.symplectic_steppers import PositionVerlet
from collections import defaultdict

In [ ]:
class JointRodSim(BaseSystemCollection, Constraints, Forcing, Damping, Connections):
    pass 

Joint_rod_sim = JointRodSim()

n_elem = 20
#-------mmGS Unit---------
density = 2.273 # mg/mm^3
gravitational_acc = -9.80655e3 #mm/s
softer_length = 3 #mm
base_radius = 0.3 #mm
ss_constant = 1 # stiffer_length / softer_length
stiffer_length = ss_constant*softer_length
E = 1.4e9 #mg / (mm*s^2)
#-------------------------
dl = softer_length/n_elem 
dt = 1e-6 
nu = 15
shear_modulus = E/3 
#---------- rod definite------
start = np.array([0.0,0.0,0.0])
direction = np.array([0.0,1.0,0.0])
normal = np.array([1.0,0.0,0.0])

softer_rod = CosseratRod.straight_rod(
    n_elem, start, direction, normal, softer_length, base_radius, density, youngs_modulus=E, shear_modulus=shear_modulus
)
stiffer_rod = CosseratRod.straight_rod(
    n_elem, start, direction, normal, stiffer_length, base_radius, density, youngs_modulus=10*E, shear_modulus= shear_modulus*10
)

Joint_rod_sim.append(softer_rod)
Joint_rod_sim.append(stiffer_rod)
#------------Damper setting------------
Joint_rod_sim.dampen(softer_rod).using(
    AnalyticalLinearDamper, damping_constant=nu, time_step = dt
)
Joint_rod_sim.dampen(stiffer_rod).using(
    AnalyticalLinearDamper, damping_constant=nu, time_step = dt
)
#-----------Constrain----------------
Joint_rod_sim.constrain(stiffer_rod).using(
    OneEndFixedRod,constraints_position_idx=(0,),constraints_director_idx = (0,)
)
Joint_rod_sim.connect(
    first_rod = stiffer_rod, second_rod = softer_rod, first_connect_idx= -1, second_connect_idx= 0
).using(FixedJoint, k=1e5, nu=0.0, kt=1e1, nut=0)

#---------Force--------
origin_force = np.array([0.0,0.0,0.0])
end_force = np.array([0.0,0.0,-10.0])
Joint_rod_sim.add_forcing_to(softer_rod).using(EndpointForces, origin_force, end_force, ramp_up_time = 0.1)
#-------callback function----
class JointRodCallBack(CallBackBaseClass):
    def __init__(self, step_skip:int, callback_params:dict):
        super.__init__(self)
        self.every = step_skip
        self.callback_params = callback_params
    def make_callback(self, system, time, current_step: int):
        if current_step%self.every == 0:
            self.callback_params["time"].append(time)
            self.callback_params["position"].append(system.position_collection.copy())
            self.callback_params["velocity"].append(system.velocity_collection.copy())

            return
pp_list = defaultdict(list)
Joint_rod_sim.collect_diagnostics(softer_rod).using(
    JointRodCallBack, step_skip = 100, callback_params= pp_list
)
Joint_rod_sim.collect_diagnostics(stiffer_rod).using(
    JointRodCallBack, step_skip = 100, callback_params = pp_list
)
#-----------time integration-------------
timestepper = PositionVerlet()
final_time = 2.0 
total_steps = final_time/ dt
integrate(timestepper, Joint_rod_sim, final_time, total_steps)